In [17]:
import torch
import torchvision.models as models
from torchvision import transforms
from custom_transforms import ImgMask
from dataset import FacialImageData
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
resnet50 = models.resnet50()
num_classes = 7 ## how many classes in fer?
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, num_classes)
pretrained = torch.load("./saved_model/resnet50_on_FER.pth")
resnet50.load_state_dict(pretrained["state_dict"])
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
resnet50.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
## mask upper val transform
mask_upper_val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        ImgMask([1, 2]),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        )
    ]
)
## mask lower val transform
mask_lower_val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        ImgMask([3, 4]),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        )
    ]
)
## mask left val transform
mask_left_val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        ImgMask([2, 3]),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        )
    ]
)
## mask right val transform
mask_right_val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        ImgMask([1, 4]),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        )
    ]
)

In [12]:
## create dataset

mask_upper_val_set = FacialImageData(
    directory="./data/test",
    transform=mask_upper_val_transform
)

mask_lower_val_set = FacialImageData(
    directory="./data/test",
    transform=mask_lower_val_transform
)

mask_left_val_set = FacialImageData(
    directory="./data/test",
    transform=mask_left_val_transform
)

mask_right_val_set = FacialImageData(
    directory="./data/test",
    transform=mask_right_val_transform
)

In [15]:
## create validation loader

VAL_BATCH_SIZE = 100
mask_upper_val_loader = DataLoader(
    mask_upper_val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=2
)
mask_lower_val_loader = DataLoader(
    mask_lower_val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=2
)
mask_left_val_loader = DataLoader(
    mask_left_val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=2
)
mask_right_val_loader = DataLoader(
    mask_right_val_set,
    batch_size=VAL_BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

In [19]:
## validate
loss_func = nn.CrossEntropyLoss().to(device)
set_names = ["upper mask", "lower mask", "left mask", "right mask"]
loaders = [mask_upper_val_loader, mask_lower_val_loader, mask_left_val_loader, mask_right_val_loader]

for i in range(len(set_names)):
    print(set_names[i])
    with torch.no_grad():
        val_loss = 0
        total_examples = 0
        correct_examples = 0
        for batch_idx, (inputs, targets) in enumerate(loaders[i]):
            # copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device)
            # compute the output and loss
            out = resnet50(inputs)
            loss = loss_func(out, targets)
            # count the number of correctly predicted samples
            # in the current batch
            _, predicted = torch.max(out, 1)
            correct = predicted.eq(targets).sum()
            val_loss += loss.detach().cpu()
            total_examples += targets.shape[0]
            correct_examples += correct.item()
    avg_loss = val_loss / len(loaders[i])
    avg_acc = correct_examples / total_examples
    print(
        "Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc)
    )

upper mask
Validation loss: 1.9188, Validation accuracy: 0.4489
lower mask
Validation loss: 2.0276, Validation accuracy: 0.2753
left mask
Validation loss: 1.6110, Validation accuracy: 0.4388
right mask
Validation loss: 1.7411, Validation accuracy: 0.4101
